# 03 - Modelo completo (SVM y comparación)
Este notebook integra el preprocesamiento realizado en `02 - preprocesado.ipynb`, selecciona la variable objetivo y entrena varios modelos (SVM principal + comparación). Generado automáticamente en base a tus notebooks 01 y 02. Ajusta los nombres de archivo/columnas si es necesario.

## Resumen automático de los notebooks leídos
Extraído automáticamente:

**Archivos encontrados en preprocesado:**

train.csv
test.csv

**Nombres de DataFrame detectados en preprocesado:**

X_num, X_num, df_col, df_train, df_test

**Columnas detectadas en preprocesado (candidatas a target):**

No se detectó una columna objetivo obvia.

## 1) Cargar datos preprocesados
Si en el notebook 02 guardaste el DataFrame a un archivo CSV, reemplaza `PREPROCESADO_FILE` por ese nombre.
Si no guardaste un CSV, en su lugar puedes volver al notebook 02 y ejecutar la última celda que produce el DataFrame y exportarlo con `df.to_csv('mi_preprocesado.csv', index=False)`.

In [ ]:
# Ajusta este nombre al archivo que hayas generado en el notebook 02
PREPROCESADO_FILE = 'datos_preprocesados.csv'  # <-- reemplaza si tu archivo tiene otro nombre

import pandas as pd
df = pd.read_csv(PREPROCESADO_FILE)
print('Shape:', df.shape)
df.head()

## 2) Separar características y variable objetivo (no detectada automáticamente)
Por favor, cambia `TARGET_COL` por la columna que quieras predecir.

In [ ]:
# Cambia 'target_column' por la columna real que quieres predecir
TARGET_COL = 'target_column'
X = df.drop(columns=[TARGET_COL])
y = df[TARGET_COL]
X.shape, y.shape

## 3) División entrenamiento/prueba

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y if len(y.unique())>1 else None)
X_train.shape, X_test.shape

## 4) Pipeline y modelos a probar
Entrenaremos:
- SVM (principal)
- Random Forest
- XGBoost (si está instalado)
Realizaremos búsqueda rápida de hiperparámetros con GridSearchCV para SVM.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

pipe_svc = Pipeline([('scaler', StandardScaler()), ('svc', SVC(probability=True))])
pipe_rf = Pipeline([('rf', RandomForestClassifier(random_state=42))])

# Parámetros para búsqueda
param_svc = {
    'svc__kernel': ['rbf','linear'],
    'svc__C': [0.1,1,10],
    'svc__gamma': ['scale','auto']
}

param_rf = {
    'rf__n_estimators': [100,200],
    'rf__max_depth': [None, 10, 20]
}

gs_svc = GridSearchCV(pipe_svc, param_svc, cv=3, scoring='accuracy', n_jobs=-1)
gs_rf = GridSearchCV(pipe_rf, param_rf, cv=3, scoring='accuracy', n_jobs=-1)

print('Modelos y búsquedas preparadas')

### 4.1 Entrenar SVM (Grid Search)

In [ ]:
gs_svc.fit(X_train, y_train)
print('Mejores parámetros SVM:', gs_svc.best_params_)
y_pred_svc = gs_svc.predict(X_test)
print('Accuracy SVM:', accuracy_score(y_test, y_pred_svc))
print(classification_report(y_test, y_pred_svc))

### 4.2 Entrenar Random Forest (Grid Search)

In [ ]:
gs_rf.fit(X_train, y_train)
print('Mejores parámetros RF:', gs_rf.best_params_)
y_pred_rf = gs_rf.predict(X_test)
print('Accuracy RF:', accuracy_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))

### 4.3 Intento de XGBoost (opcional)

In [ ]:
try:
    from xgboost import XGBClassifier
    xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
    xgb.fit(X_train, y_train)
    y_pred_xgb = xgb.predict(X_test)
    print('Accuracy XGB:', accuracy_score(y_test, y_pred_xgb))
    print(classification_report(y_test, y_pred_xgb))
except Exception as e:
    print('XGBoost no disponible o falló al entrenar:', e)

## 5) Importancia de variables y explicación (si aplica)
- Para RandomForest/XGBoost puedes mostrar `feature_importances_`.
- Para SVM puedes usar técnicas como permutation importance o SHAP.

In [ ]:
# Importancia simple para RandomForest si existe
try:
    import numpy as np
    rf_best = gs_rf.best_estimator_.named_steps['rf']
    importances = rf_best.feature_importances_
    feat_imp = pd.Series(importances, index=X.columns).sort_values(ascending=False).head(20)
    feat_imp
except Exception as e:
    print('No se pudo calcular importancias con RF:', e)

## 6) Guardar modelo y resultados
Guarda el mejor modelo y los resultados para entregar.

In [ ]:
import joblib
joblib.dump(gs_svc.best_estimator_, 'modelo_svc_best.joblib')
joblib.dump(gs_rf.best_estimator_, 'modelo_rf_best.joblib')
print('Modelos guardados: modelo_svc_best.joblib, modelo_rf_best.joblib')

## Notas finales
- Revisa las celdas donde hay que reemplazar nombres (archivo preprocesado y columna target).
- Si quieres que yo ejecute esto con tus datos y te devuelva el notebook ya con resultados, puedo intentarlo (si hay permisos y recursos).